# Library

In [1]:
import pandas as pd
import numpy as np
import os

from numpy import arange
import openpyxl
from io import BytesIO

 
import matplotlib.pyplot as plt  
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
%matplotlib inline


import itertools
import statsmodels.api as sm


from sklearn.metrics import r2_score


from tqdm import tqdm 
tqdm.pandas()




import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX,SARIMAXResults
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from statsmodels.tools.eval_measures import rmse
from statsmodels.stats.stattools import durbin_watson


import matplotlib.pyplot as plt
import seaborn as sns




from feature_engine.transformation import ReciprocalTransformer
from sklearn.preprocessing import StandardScaler ,RobustScaler, QuantileTransformer ,PowerTransformer,MaxAbsScaler,MinMaxScaler,Normalizer
from sklearn.linear_model import LinearRegression,Ridge,RidgeCV,LarsCV,ridge_regression
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_percentage_error as MAPE,mean_squared_error

from scipy.stats import zscore,iqr
from scipy.stats.mstats import winsorize


import streamlit as st

# Folder and File Path

In [2]:
# Folder Path

Folder_path=r"D:\Pranto\Working\Forward Looking PD\Mar26 Prediction\V2\Data\SCV"

# file Name

File_name=r"SCV Raw Data 01042025 V1"


#Sheet Name


# sheet_name="None"
sheet_name="SCV_FTU_BKT0"



# Export file Name 

name=r"SCV FTU BKT0"


#################################################################################
##################################################################################

# # Test File Path

# test_path=rf"{Folder_path}\Independent Variables_Train Test.xlsx"



# # Test Data

# df_test=pd.read_excel(test_path,sheet_name="test")


# print(df_test.columns)

# df_test


#################################################################################
##################################################################################

# Import  Train Data

In [3]:

# Read Data
if sheet_name=="None":
    df=pd.read_excel(rf"{Folder_path}\{File_name}.xlsx")

else:
    df=pd.read_excel(rf"{Folder_path}\{File_name}.xlsx",sheet_name=sheet_name)

df_date=df['Date']

print("Shape of the dataset",df.shape)
df.tail(5)

Shape of the dataset (55, 44)


,Date,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),Brent,...,"Private consumption, contribution to real GDP growth (% points)",Real effective exchange rate (index; 2010=100),Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),Bank_Rate/RR,PD_BKT_0,PD_BKT_0_Mean,PD_BKT_0_Near
50,2024-11-01,184.1,0.0620,147.0,133.8,6.5,0.0548,148.1,4.96,74.350,...,3.7,104.4,6.3,184705.4,3.1,8.5,1.038462,3.011939,3.011939,3.011939
51,2024-12-01,192.8,0.0620,156.8,143.2,6.5,0.0522,157.7,3.55,73.860,...,3.7,104.4,6.3,184705.4,3.1,8.5,1.038462,3.197129,3.197129,3.197129
52,2025-01-01,201.9,0.0564,159.1,150.7,6.5,0.0426,161.3,5.01,79.270,...,3.8,104.8,6.5,196785.1,3.3,7.0,1.038462,3.229318,3.229318,3.229318
53,2025-02-01,204.8,0.0564,153.5,135.2,6.5,0.0361,150.9,3.27,75.440,...,3.8,104.8,6.5,196785.1,3.3,7.0,1.038462,3.273647,3.273647,3.273647
54,2026-03-31,215.0,0.0598,160.2,158.0,6.5,0.0416,164.3,4.39,72.766,...,3.8,107.6,6.7,209910.7,3.5,7.4,1.038462,NaN,NaN,NaN


In [4]:

df.columns

Index(['Date', 'Electricity ', 'GDP', 'Manufacturing ', 'Mining ', 'RR', 'CPI',
       'IIP', 'IIP (YOY)', 'Brent', 'PMI', 'WPI', 'Bank_Rate', 'Diesel',
       'Sales_Commerical_Vehicle', 'Sales_Passenger_Vehicles', 'Seasonality',
       'Quarter', 'Unnamed: 19', 'Construction (% of GDP)',
       'Consumer prices (% change pa; av)',
       'Current-account balance (% of GDP)', 'Domestic credit growth (%)',
       'Exports of G&S (% of GDP)', 'GDP deflator (% change; av)',
       'Gross fixed investment (% of GDP)',
       'Gross national savings/investment (%)', 'Imports of G&S (% of GDP)',
       'Lending interest rate (%)', 'Long-term bond yield (%)',
       'Nominal GDP (LCU)', 'Policy interest rate (%)',
       'Private consumption (% of GDP)',
       'Private consumption (real % change pa)',
       'Private consumption, contribution to real GDP growth (% points)',
       'Real effective exchange rate (index; 2010=100)',
       'Real GDP (% change pa)', 'Real GDP (LCU)',
       'To

In [5]:

###############################################################################
##################################################################################
# Choose Dependent And Independent Variables


# Dependent Variables
col_dependent='PD_BKT_0'
# col_dependent='PD_BKT_0_Mean'
# col_dependent='PD_BKT_0_Near'


# Independent Variables

col_independent=[f'{col_dependent}_Lag',
                'Electricity ',
                'GDP',
                'Manufacturing ',
                'Mining ',
                'RR',
                'CPI',
                'IIP',
                'IIP (YOY)',
                'Brent',
                'PMI',
                'WPI',
                'Bank_Rate',
                'Bank_Rate/RR',
                'Diesel',
                'Sales_Commerical_Vehicle',
                'Sales_Passenger_Vehicles',
                'Seasonality',
                'Quarter',
                'Construction (% of GDP)',
                'Consumer prices (% change pa; av)',
                'Current-account balance (% of GDP)', 
                'Domestic credit growth (%)',
                'Exports of G&S (% of GDP)', 'GDP deflator (% change; av)',
                'Gross fixed investment (% of GDP)',
                'Gross national savings/investment (%)', 
                'Imports of G&S (% of GDP)',
                'Lending interest rate (%)',
                'Long-term bond yield (%)',
                'Nominal GDP (LCU)', 'Policy interest rate (%)',
                'Private consumption (% of GDP)',
                'Private consumption (real % change pa)',
                'Private consumption, contribution to real GDP growth (% points)',
                'Real effective exchange rate (index; 2010=100)',
                'Real GDP (% change pa)', 'Real GDP (LCU)',
                'Total factor productivity growth (%)', 'Unemployment rate (%)'
                # ,'Housing Price Index (% change)'
]


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")


print("Dependent variable = ",col_dependent)


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

################################################################################
##################################################################################

# Outlier Treatement

col_dep=col_dependent # Column's name from you want to remove or replace outliers 
num=2          # Z score value
percentile=0.95  # Percentile


print(f"Outlier Treatement in Column {col_dep}")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")


################################################################################
##################################################################################
# Select Below Method

# method="None"
method="Mean Standard Deviation method"
# method="IQR Method"
# method="Percentile Method"


print(f"Seleceted Outlier Method = ",method)


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

##################################################################################
##################################################################################

# Log and Lag variable Creation


lag=1  # Lag value

col_log=col_dependent  # Log Column

col_lag=col_dependent  # Lag Column


print(f"Selected column {col_lag}_Lag for Lag and {col_log}_Log(10) for creating Log Variable  and number of Lag = {lag} ")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

##################################################################################
##################################################################################

# Remove Number of Rows from Last for test data

number=1


print(f"Remove last {number} of rows from last for test data")


print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")
print("\n")

#################################################################################
##################################################################################

# Standarization of Variables


method_std='Standard Scaler'
# method_std='Log Transformation'
# method_std='Yeo-Johnson'
# method_std='Robust Scaler'
# method_std='Quantile Transformer'
# method_std='Max Abs Scaler'
# method_std='MinMax Scaler'
# method_std='Box-Cox Transformer'
# method_std='Reciprocal Transformer'
# method_std='Sin Transformation'
# method_std='Cos transformation'
# method_std='Tan Transformation'



col=col_independent

# col=['Electricity ',
#                 'GDP',
#                 'Manufacturing ',
#                 'Mining ',
#                 'RR',
#                 'CPI',
#                 'IIP',
#                 'IIP (YOY)',
#                 'Brent',
#                 'PMI',
#                 'WPI',
#                 'Bank_Rate',
#                 'Bank_Rate/RR',
#                 'Diesel',
#                 'Sales_Commerical_Vehicle',
#                 'Sales_Passenger_Vehicles',
#                 'Seasonality',
#                 'Quarter']


print(f"Selected standarization method : {method_std}")

# print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")



###############################################################################
##################################################################################

# Conditions For Possible Combinations

# Dependent Variable

Y1=col_dependent




# Others Parameter


adj_r2_num=0.6 # For Adjusted R2

vif_limit=5 # Vif Limit

dw_upper=2.5 # Durbin Watson Upper limit

dw_lower=1.5  # Durbin Watson Lower limit


p_val=0.05

comb_lower=2 # Lower Combination 
comb_upper=4 # Upper Combination


comb_upper=comb_upper+1

# df_itteration=df

delete_row=number  # Delete row from last for testing


################################################################################
##################################################################################

desired_directions={
    'Electricity ':'Negative',
    'GDP':'Negative',
    'Manufacturing ':'Negative',
    'Mining ':'Negative',
    'RR': 'Positive',
    'CPI': 'Positive',
    'IIP':'Negative',
    'IIP (YOY)':'Negative',
    'Brent': 'Positive',
    'PMI':'Negative',
    'WPI': 'Positive',
    'Bank_Rate': 'Positive',
    'Bank_Rate/RR': 'Positive',
    'Diesel': 'Positive',
    'Sales_Commerical_Vehicle':'Negative',
    'Sales_Passenger_Vehicles':'Negative',
    'Seasonality':'Both',
    'Quarter':'Both',
     
    'Construction (% of GDP)' : 'Negative',
    'Consumer prices (% change pa; av)' : 'Positive',
    'Current-account balance (% of GDP)' : 'Negative',
    'Domestic credit growth (%)' : 'Positive',
    'Exports of G&S (% of GDP)' : 'Negative',
    'GDP deflator (% change; av)' : 'Positive',
    'Gross fixed investment (% of GDP)' : 'Negative',
    'Gross national savings/investment (%)' : 'Negative',
    'Imports of G&S (% of GDP)' : 'Negative',
    'Lending interest rate (%)' : 'Positive',
    'Long-term bond yield (%)' : 'Positive',
    'Nominal GDP (LCU)' : 'Negative',
    'Policy interest rate (%)' : 'Positive',
    'Private consumption (% of GDP)' : 'Negative',
    'Private consumption (real % change pa)' : 'Negative',
    'Private consumption, contribution to real GDP growth (% points)' : 'Negative',
    'Real effective exchange rate (index; 2010=100)' : 'Positive',
    'Real GDP (% change pa)' : 'Negative',
    'Real GDP (LCU)' : 'Negative',
    'Total factor productivity growth (%)' : 'Negative',
    'Unemployment rate (%)' : 'Positive',
    'Housing Price Index (% change)' : 'Negative'
    
 
}




&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Dependent variable =  PD_BKT_0
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Outlier Treatement in Column PD_BKT_0
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Seleceted Outlier Method =  Mean Standard Deviation method
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Selected column PD_BKT_0_Lag for Lag and PD_BKT_0_Log(10) for creating Log Variable  and number of Lag = 1 
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Remove last 1 of rows from last for test data
&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&


Selected standarization method : Standard Scaler


# Outliers Treatment

In [6]:


# By Mean Standard Deviation method

if method=="Mean Standard Deviation method":

    df00=df
    df00=df00.drop('Date',axis=1)


    upper_limit0= df00[col_dep].mean() + num*df00[col_dep].std()
    lower_limit0 = df00[col_dep].mean() - num*df00[col_dep].std()

    Q1 = df00[col_dep].quantile(q=round((1-percentile),2))
    Q4 = df00[col_dep].quantile(q=percentile)

    # upper_limit0.values

    upper_limit=min(upper_limit0,Q4)
    lower_limit=max(lower_limit0,Q1)

    # upper_limit=min(float(upper_limit0),float(Q4))
    # lower_limit=max(float(lower_limit0[col]),float(Q1))

    df00_nonzero=df00[df00[col_dep]!=0]
    mean=np.mean(df00_nonzero)
    median=df00_nonzero.median()


    df00[col_dep]=df00[col_dep].replace(0,mean)

    lower_outlier=df00[df00[col_dep]< lower_limit0]
    upper_outlier=df00[df00[col_dep]> upper_limit0]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    # a=upper_outlier[upper_outlier[col].isna()].index.to_list()
    # a
    # print("**Lower Limit and Location of Outliers**")
    print("**Lower Limit :**",lower_limit0,"**Lower Quantile**",Q1,"**Final Lower Limit**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit0,"**Upper Quantile**",Q4,"**Final Upper Limit**",upper_limit)
    # Q4
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)



    print("Replace Outliers by Percntile_Upper_Lower Limit")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit0, upper_limit,
    np.where(df00[col_dep]< lower_limit0, lower_limit,
        df00[col_dep]))

    df=df00

    print(df.shape)
    df.head()


if method=="Percentile Method":

    df00=df
    df00=df00.drop('Date',axis=1)

    
    lower_limit0=Q1
    upper_limit0=Q4

    upper_limit=Q4
    lower_limit=Q1



    lower_outlier=df00[df00[col_dep]< lower_limit0]
    upper_outlier=df00[df00[col_dep]> upper_limit0]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    # a=upper_outlier[upper_outlier[col].isna()].index.to_list()
    # a
    # print("**Lower Limit and Location of Outliers**")
    print("**Lower Limit :**",lower_limit0,"**Lower Quantile**",Q1,"**Final Lower Limit**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit0,"**Upper Quantile**",Q4,"**Final Upper Limit**",upper_limit)
    # Q4
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)



    print("Replace Outliers by Percntile_Upper_Lower Limit")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit0, upper_limit,
    np.where(df00[col_dep]< lower_limit0, lower_limit,
        df00[col_dep]))

    df=df00

    print(df.shape)
    df.head()


if method=="IQR Method":

    df00=df
    df00=df.drop('Date',axis=1)

    df00_nonzero=df00[df00[col_dep]!=0]
    mean=np.mean(df00_nonzero[col_dep])
    median=np.median(df00_nonzero)
    #find Q1, Q3, and interquartile range for each column
    Q1 = df00[col_dep].quantile(q=0.25)
    Q3 = df00[col_dep].quantile(q=0.75)
    # IQR = df00[col].apply(iqr)
    IQR=Q3-Q1

    lower_limit=Q1-1.5*IQR
    upper_limit=Q3+1.5*IQR

    
    lower_outlier=df00[df00[col_dep]< lower_limit]
    upper_outlier=df00[df00[col_dep]> upper_limit]


    loc_lower_outlier = lower_outlier[col_dep][lower_outlier[col_dep].notna()]
    loc_upper_outlier = upper_outlier[col_dep][upper_outlier[col_dep].notna()]

    
    print("**Lower Limit :**",lower_limit)
    print("**Number of Lower Outliers :**",loc_lower_outlier.notna().sum())
    print("**Lower Outlier Location**",loc_lower_outlier)

    print("**Upper Limit :**",upper_limit)
    print("**Number of Upper Outliers :**",loc_upper_outlier.notna().sum())
    print("**Upper Outlier Location**",loc_upper_outlier)
    
    

    print("**Want to replace Outlier**")

    df00[col_dep]=df00[col_dep].replace(0,mean)

    df00[col_dep] = np.where(df00[col_dep]> upper_limit, upper_limit,
    np.where(df00[col_dep]< lower_limit, lower_limit,
        df00[col_dep]))
    

    
    df=df00

    print(df.shape)

    df.head()
    



**Lower Limit :** -0.3509175471416821 **Lower Quantile** 1.7899649730168297 **Final Lower Limit** 1.7899649730168297
**Number of Lower Outliers :** 0
**Lower Outlier Location** Series([], Name: PD_BKT_0, dtype: float64)
**Upper Limit :** 9.842710074876095 **Upper Quantile** 8.698137717839302 **Final Upper Limit** 8.698137717839302
**Number of Upper Outliers :** 2
**Upper Outlier Location** 15    12.564300
16    14.375524
Name: PD_BKT_0, dtype: float64
Replace Outliers by Percntile_Upper_Lower Limit
(55, 43)


# Log and Lag variable 

In [7]:
df[col_log+"_Log(10)"]=np.log10(df[col_log])  # Create Log column

df[col_lag+"_Lag"]=df[col_lag].shift(periods=lag) # create lag column

df=df.iloc[lag:] #delete number of lag
                    
                    
print(df.shape)
df.head()

                    

(54, 45)


,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),Brent,PMI,...,Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),Bank_Rate/RR,PD_BKT_0,PD_BKT_0_Mean,PD_BKT_0_Near,PD_BKT_0_Log(10),PD_BKT_0_Lag
1,162.2,0.004611,132.0,98.5,4.0,0.076087,129.6,4.52,40.19,58.9,...,-5.962,136527.86,-5.5,22.7,1.0625,0.825157,0.825157,0.825157,-0.083464,1.018042
2,144.8,0.004611,128.5,106.6,4.0,0.069314,126.7,-1.63,42.69,56.3,...,-5.962,136527.86,-5.5,22.7,1.0625,0.852468,0.852468,0.852468,-0.069322,0.825157
3,158.0,0.004611,139.0,117.3,4.0,0.045878,137.4,2.16,49.99,56.4,...,-5.962,136527.86,-5.5,22.7,1.0625,2.675295,2.675295,2.675295,0.427372,0.852468
4,164.2,0.016417,136.6,121.3,4.0,0.040613,136.6,-0.58,54.77,57.7,...,9.842,149964.90,6.3,20.5,1.0625,2.369545,2.369545,2.369545,0.374665,2.675295
5,153.9,0.016417,129.7,117.9,4.0,0.050302,129.9,-3.20,62.28,57.5,...,9.842,149964.90,6.3,20.5,1.0625,2.286726,2.286726,2.286726,0.359214,2.369545


# Delete Rows From last

In [8]:
# Delete number of rows from last




df0=df.tail(int(number))
df=df.head(-int(number))





print("Remove rows shape",df0.shape)

print("Rest Data shape",df.shape)

df.head()

Remove rows shape (1, 45)
Rest Data shape (53, 45)


,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),Brent,PMI,...,Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),Bank_Rate/RR,PD_BKT_0,PD_BKT_0_Mean,PD_BKT_0_Near,PD_BKT_0_Log(10),PD_BKT_0_Lag
1,162.2,0.004611,132.0,98.5,4.0,0.076087,129.6,4.52,40.19,58.9,...,-5.962,136527.86,-5.5,22.7,1.0625,0.825157,0.825157,0.825157,-0.083464,1.018042
2,144.8,0.004611,128.5,106.6,4.0,0.069314,126.7,-1.63,42.69,56.3,...,-5.962,136527.86,-5.5,22.7,1.0625,0.852468,0.852468,0.852468,-0.069322,0.825157
3,158.0,0.004611,139.0,117.3,4.0,0.045878,137.4,2.16,49.99,56.4,...,-5.962,136527.86,-5.5,22.7,1.0625,2.675295,2.675295,2.675295,0.427372,0.852468
4,164.2,0.016417,136.6,121.3,4.0,0.040613,136.6,-0.58,54.77,57.7,...,9.842,149964.90,6.3,20.5,1.0625,2.369545,2.369545,2.369545,0.374665,2.675295
5,153.9,0.016417,129.7,117.9,4.0,0.050302,129.9,-3.20,62.28,57.5,...,9.842,149964.90,6.3,20.5,1.0625,2.286726,2.286726,2.286726,0.359214,2.369545


# Dependent And Independent Variable

In [9]:
df.columns

Index(['Electricity ', 'GDP', 'Manufacturing ', 'Mining ', 'RR', 'CPI', 'IIP',
       'IIP (YOY)', 'Brent', 'PMI', 'WPI', 'Bank_Rate', 'Diesel',
       'Sales_Commerical_Vehicle', 'Sales_Passenger_Vehicles', 'Seasonality',
       'Quarter', 'Unnamed: 19', 'Construction (% of GDP)',
       'Consumer prices (% change pa; av)',
       'Current-account balance (% of GDP)', 'Domestic credit growth (%)',
       'Exports of G&S (% of GDP)', 'GDP deflator (% change; av)',
       'Gross fixed investment (% of GDP)',
       'Gross national savings/investment (%)', 'Imports of G&S (% of GDP)',
       'Lending interest rate (%)', 'Long-term bond yield (%)',
       'Nominal GDP (LCU)', 'Policy interest rate (%)',
       'Private consumption (% of GDP)',
       'Private consumption (real % change pa)',
       'Private consumption, contribution to real GDP growth (% points)',
       'Real effective exchange rate (index; 2010=100)',
       'Real GDP (% change pa)', 'Real GDP (LCU)',
       'Total fact

In [10]:
# Dependent Variables



# Dependent 
y=df[col_dependent]


# Independent Variables
x=col_independent

# Independent Dataframe
x=df[x]

#Test data
x_test=df0[col_independent]


print("Independ Variable Dataset Shape",x.shape)
print("Dependent Variable Dataframe Shape",y.shape)
print("Test Dataframe Shape",x_test.shape)

Independ Variable Dataset Shape (53, 40)
Dependent Variable Dataframe Shape (53,)
Test Dataframe Shape (1, 40)


# Standarization

In [11]:
if method_std=='Log Transformation':
    x[col]=np.log(x[col])
    x_test[col]=np.log(x_test[col])
    
    print(x.shape)

    # print(x.describe().T)


if method_std=='Standard Scaler':
    std_scaler=StandardScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    # x=std_scaler.fit_transform(x[col])
    # x=pd.DataFrame(x,columns=col)
    # x_test=std_scaler.transform(x_test[col])
    # x_test=pd.DataFrame(x_test,columns=col)

    
        
    print(x.shape)

    # Calculate mean and std for each column
    mean_values = std_scaler.mean_
    std_values = std_scaler.scale_

    # Print the mean and std values in tabular form
    summary_df = pd.DataFrame({
        'Column': col,
        'Mean': mean_values,
        'Std': std_values
    })
    
    print("Mean and Std of each column after scaling:")
    print(summary_df.head(5))

    


if method_std=='Normalizer':
    std_scaler=Normalizer()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    
    
    print(x.shape)

    
    


if method_std=='Robust Scaler':
    std_scaler=RobustScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    
    


if method_std=='Quantile Transformer':
    std_scaler=QuantileTransformer()
    x[col]=std_scaler.fit_transform(x[col])

   
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    

if method_std=='Max Abs Scaler':
    std_scaler=MaxAbsScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='MinMax Scaler':
    std_scaler=MinMaxScaler()
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='Box-Cox Transformer':
    std_scaler=PowerTransformer(method='box-cox')
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    


if method_std=='Yeo-Johnson':
    std_scaler=PowerTransformer(method='yeo-johnson')
    x[col]=std_scaler.fit_transform(x[col])
    x_test[col]=std_scaler.transform(x_test[col])

    print(x.shape)

    

if method_std=='Reciprocal Transformer':
    x[col]=1/x[col]
    x_test[col]=1/x_test[col]

    print(x.shape)

    
    # pass
if method_std=='Sin Transformation':
    x[col]=np.sin(x[col])
    x_test[col]=np.sin(x_test[col])

    print(x.shape)

    
    # pass
if method_std=='Cos transformation':
    x[col]=np.cos(x[col])
    x_test[col]=np.cos(x_test[col])

    print(x.shape)

    
    # pass
if method_std=='Tan Transformation':
    x[col]=np.tan(x[col])
    x_test[col]=np.tan(x_test[col])

    print(x.shape)

    
            

(53, 40)
Mean and Std of each column after scaling:
           Column        Mean        Std
0    PD_BKT_0_Lag    4.593608   2.028982
1    Electricity   186.830189  20.084037
2             GDP    0.071192   0.042240
3  Manufacturing   139.724528   9.221957
4         Mining   122.143396  15.121277


C:\Users\524264\AppData\Local\Temp\ipykernel_24960\242110819.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[col]=std_scaler.fit_transform(x[col])
C:\Users\524264\AppData\Local\Temp\ipykernel_24960\242110819.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[col]=std_scaler.transform(x_test[col])


#  Model Data

In [12]:
from datetime import datetime

x1=pd.concat([x,x_test],axis=0)

# df_date=df_date.dt.strftime("%b/%y")

model_data=pd.concat([df_date,x1,y],axis=1)



# Drop rows with missing values except in the 'Date' column
model_data = model_data.dropna(subset=[col for col in model_data.columns if col != 'Date'],how='all')



print(model_data.shape)
model_data.tail(3)


(54, 42)


,Date,PD_BKT_0_Lag,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),...,Policy interest rate (%),Private consumption (% of GDP),Private consumption (real % change pa),"Private consumption, contribution to real GDP growth (% points)",Real effective exchange rate (index; 2010=100),Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),PD_BKT_0
52,2025-01-01,-0.688266,0.750338,-0.350194,2.101015,1.888505,0.915409,-1.233485,2.088318,-0.160434,...,0.039143,-0.673294,0.00801,0.01064,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.229318
53,2025-02-01,-0.672401,0.894731,-0.350194,1.493769,0.863459,0.915409,-1.835735,1.024669,-0.255159,...,0.039143,-0.673294,0.00801,0.01064,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.273647
54,2026-03-31,-0.650553,1.402597,-0.269702,2.220296,2.371268,0.915409,-1.326139,2.395140,-0.194186,...,-1.574419,-1.673073,0.00801,0.01064,4.913139,-0.091319,2.783203,-0.210564,-1.089386,NaN


# Export Model Data

In [13]:

# model_data.to_excel(rf"{Folder_path}\{File_name} Model Data.xlsx",sheet_name=f"{File_name}",index=False)



# Import Data

In [14]:
# df = pd.read_excel(r"C:\Users\524264\Downloads\OUTLIER_REPLACED_SCV_FTU_BKT_0.xlsx")
# df=df.tail(-int(1))


df=model_data


# Delete 1st 1 columns Date

df=df.iloc[:,1:]
# df=df.iloc[:,1:]


print(df.shape)
df.tail()
# df.tail()
# DATA.shape

(54, 41)


,PD_BKT_0_Lag,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),Brent,...,Policy interest rate (%),Private consumption (% of GDP),Private consumption (real % change pa),"Private consumption, contribution to real GDP growth (% points)",Real effective exchange rate (index; 2010=100),Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),PD_BKT_0
50,-0.717117,-0.135938,-0.217619,0.788929,0.770874,0.915409,-0.103108,0.738302,-0.163156,-0.427222,...,0.730670,-1.423128,0.058101,-0.033381,1.409067,-0.207873,1.163330,-0.364493,-0.890104,3.011939
51,-0.779538,0.297242,-0.217619,1.851610,1.392515,0.915409,-0.344008,1.720132,-0.239916,-0.458005,...,0.730670,-1.423128,0.058101,-0.033381,1.409067,-0.207873,1.163330,-0.364493,-0.890104,3.197129
52,-0.688266,0.750338,-0.350194,2.101015,1.888505,0.915409,-1.233485,2.088318,-0.160434,-0.118140,...,0.039143,-0.673294,0.008010,0.010640,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.229318
53,-0.672401,0.894731,-0.350194,1.493769,0.863459,0.915409,-1.835735,1.024669,-0.255159,-0.358747,...,0.039143,-0.673294,0.008010,0.010640,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.273647
54,-0.650553,1.402597,-0.269702,2.220296,2.371268,0.915409,-1.326139,2.395140,-0.194186,-0.526732,...,-1.574419,-1.673073,0.008010,0.010640,4.913139,-0.091319,2.783203,-0.210564,-1.089386,NaN


# Change the Dependent Variable and Other Parameters

In [15]:

# # Dependent Variable

# Y1='PD_BKT1'




# # Others Parameter


# adj_r2_num=0.6 # For Adjusted R2

# vif_limit=5 # Vif Limit

# dw_upper=2.5 # Durbin Watson Upper limit

# dw_lower=1.5  # Durbin Watson Lower limit


# p_val=0.05

# comb_lower=2 # Lower Combination 
# comb_upper=4 # Upper Combination


# comb_upper=comb_upper+1

# # df_itteration=df

# delete_row=1  # Delete row from last for testing



# desired_directions={
#     'Electricity ':'Negative',
#     'GDP':'Negative',
#     'Manufacturing ':'Negative',
#     'Mining ':'Negative',
#     'RR': 'Positive',
#     'CPI': 'Positive',
#     'IIP':'Negative',
#     'IIP (YOY)':'Negative',
#     'Brent': 'Positive',
#     'PMI':'Negative',
#     'WPI': 'Positive',
#     'Bank_Rate': 'Positive',
#     'Bank_Rate/RR': 'Positive',
#     'Diesel': 'Positive',
#     'Sales_Commerical_Vehicle':'Negative',
#     'Sales_Passenger_Vehicles':'Negative',
#     'Seasonality':'Both',
#     'Quarter':'Both'
# }






In [16]:
df0=df.tail(int(delete_row))

df_itteration=df.head(-int(delete_row))


print("Train Data Shape",df_itteration.shape)

print("Test Data Shape",df0.shape)

df_itteration.tail()

Train Data Shape (53, 41)
Test Data Shape (1, 41)


,PD_BKT_0_Lag,Electricity,GDP,Manufacturing,Mining,RR,CPI,IIP,IIP (YOY),Brent,...,Policy interest rate (%),Private consumption (% of GDP),Private consumption (real % change pa),"Private consumption, contribution to real GDP growth (% points)",Real effective exchange rate (index; 2010=100),Real GDP (% change pa),Real GDP (LCU),Total factor productivity growth (%),Unemployment rate (%),PD_BKT_0
49,-0.709182,1.044103,-0.217619,0.940741,0.420375,0.915409,0.573265,0.963305,-0.230117,-0.346811,...,0.730670,-1.423128,0.058101,-0.033381,1.409067,-0.207873,1.163330,-0.364493,-0.890104,3.138590
50,-0.717117,-0.135938,-0.217619,0.788929,0.770874,0.915409,-0.103108,0.738302,-0.163156,-0.427222,...,0.730670,-1.423128,0.058101,-0.033381,1.409067,-0.207873,1.163330,-0.364493,-0.890104,3.011939
51,-0.779538,0.297242,-0.217619,1.851610,1.392515,0.915409,-0.344008,1.720132,-0.239916,-0.458005,...,0.730670,-1.423128,0.058101,-0.033381,1.409067,-0.207873,1.163330,-0.364493,-0.890104,3.197129
52,-0.688266,0.750338,-0.350194,2.101015,1.888505,0.915409,-1.233485,2.088318,-0.160434,-0.118140,...,0.039143,-0.673294,0.008010,0.010640,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.229318
53,-0.672401,0.894731,-0.350194,1.493769,0.863459,0.915409,-1.835735,1.024669,-0.255159,-0.358747,...,0.039143,-0.673294,0.008010,0.010640,1.847076,-0.149596,1.939658,-0.287529,-1.161853,3.273647


# Input Section

In [17]:
# Create a DataFrame to store all the information
data = {
    "Input Section": [
        "Dependent Variable", 
        "Independent Variables", 
        "Outlier Method",
        "Outliers information",
        "percentile",
        "Lag and Log Variable", 
        "Standardization Method",
        "Train data Shape", 
        "Test Data Shape", 
        
    ],
    "Details": [
        col_dependent,  # Dependent variable
        ', '.join(col_independent),  # independent variables
        f"{method}, Outlier treatement column : {col_dep}", # Outlier methods
        f"Lower limit ==== {lower_limit}, Number of Lower Outlier ==== {loc_lower_outlier.notna().sum()},Lower Limit Location ==== {loc_lower_outlier.to_dict()},Upper Limit ==== {upper_limit},Number of total upper outlier ==== {loc_upper_outlier.notna().sum()},Upper Limit Location ==== {loc_upper_outlier.to_dict()}",
        f"5th Percentile ==== {Q1}, 95th Percentile ==== {globals().get('Q4',)}",  # 5th and 95th Percentiole
        f"Log Column: {col_log}_Log(10), Lag Value: {col_lag}_Lag", # Log and Lag column
        method_std, # Standarize method
        df_itteration.shape,  #Train Data shape
        df0.shape # Test Data shape
    ]
}

# Create the DataFrame
df_info = pd.DataFrame(data)

# Display the DataFrame
df_info


,Input Section,Details
0,Dependent Variable,PD_BKT_0
1,Independent Variables,"PD_BKT_0_Lag, Electricity , GDP, Manufacturing..."
2,Outlier Method,"Mean Standard Deviation method, Outlier treate..."
3,Outliers information,"Lower limit ==== 1.7899649730168297, Number of..."
4,percentile,"5th Percentile ==== 1.7899649730168297, 95th P..."
5,Lag and Log Variable,"Log Column: PD_BKT_0_Log(10), Lag Value: PD_BK..."
6,Standardization Method,Standard Scaler
7,Train data Shape,"(53, 41)"
8,Test Data Shape,"(1, 41)"


In [18]:
# Function to calculate VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    try:
        vif_data["VIF"] = [VIF(X.values, i) for i in range(X.shape[1])]
    except Exception as e:
        print("Error calculating VIF:", e)
        vif_data["VIF"] = np.nan
    return vif_data


# Generate all combinations of independent variables
all_vars = df_itteration.columns.drop(Y1)
combinations = []

for r in range(comb_lower, comb_upper):
    combinations.extend(itertools.combinations(all_vars, r))

print("Total Combination", len(combinations))


# Initialize lists to store results and regression summaries
results = []
regression_summaries = []

# Iterate over all combinations
for combo in tqdm(combinations, desc="Processing Time"):
    X = df_itteration[list(combo)]
    X_test = df0[list(combo)]
    X = sm.add_constant(X)  # Add constant for intercept term
    combo = list(X.columns)
    X_test.insert(0, 'const', 1)

    # Calculate VIF
    vif = [VIF(X.values, i) for i in range(X.shape[1])]

    if max(vif) <= vif_limit:
        model = sm.OLS(df_itteration[Y1], X).fit()
        if (model.pvalues <= p_val).all():  # Check if all p-values are <= 0.05
            y_pred = model.predict(X)
            r2 = round(r2_score(df_itteration[Y1], y_pred), 4)
            n = len(df_itteration)
            k = len(combo)
            adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - k - 1))
            adj_r2 = round(adj_r2, 4)

            # Calculate estimates
            lower_estimates = (X_test.mul(model.conf_int()[0])).sum(axis=1)
            predictions = (X_test.mul(model.params)).sum(axis=1)
            upper_estimates = (X_test.mul(model.conf_int()[1])).sum(axis=1)

            # Filter conditions for estimates
            if ((lower_estimates >= 0).all() and (predictions >= 0).all() and
                (upper_estimates >= 0).all() and (lower_estimates < predictions).all() and
                (predictions < upper_estimates).all()):
                sum_estimates = (np.round(lower_estimates.values + predictions.values + upper_estimates.values, 4))[0]
                if adj_r2 >= adj_r2_num:
                    dw_statistic = round(durbin_watson(model.resid), 4)
                    if dw_lower <= dw_statistic <= dw_upper:
                        bp_test = het_breuschpagan(model.resid, X)
                        bp_pvalue = round(bp_test[1], 4)

                        # Capture regression summary as text
                        reg_summary = model.summary2().as_text()

                        # Append results
                        results.append({
                            'Variables': combo,
                            'Coefficients': np.round(model.params.values, 4),
                            'Lower Estimate': np.round(lower_estimates.values, 4)[0],
                            'Estimates': np.round(predictions.values, 4)[0],
                            'Upper Estimate': np.round(upper_estimates.values, 4)[0],
                            'Sum of Estimates': sum_estimates,
                            'R-squared': np.round(r2, 4),
                            'Adjusted R-squared': np.round(adj_r2, 4),
                            'P-values': np.round(model.pvalues.values, 4),
                            'VIF': np.round(vif, 4),
                            'Durbin-Watson': np.round(dw_statistic, 4),
                            'Breusch-Pagan P Value': bp_pvalue,
                            'Mean Residuals': model.resid.mean()
                        })

                        # Append regression summary to a separate list
                        regression_summaries.append({
                            'Variables': combo,
                            'Regression Summary': reg_summary
                        })

# Create DataFrame from results
results_df = pd.DataFrame(results)

# Create a separate DataFrame for regression summaries
regression_summary_df = pd.DataFrame(regression_summaries)

# Function to check if row follows the correct direction
def check_direction(variables, coefficients):
    for var, coeff in zip(variables, coefficients):
        if var in desired_directions:
            if (coeff > 0 and desired_directions[var] == 'Negative') or \
               (coeff < 0 and desired_directions[var] == 'Positive'):
                return False
    return True

print("Result_df Shape", results_df.shape)

# Filter and sort results
try:
    results_df.sort_values(by=['Adjusted R-squared', 'Sum of Estimates', 'Durbin-Watson'], ascending=[False, True, True], inplace=True)
    results_df_filtered = results_df[results_df.progress_apply(
        lambda x: check_direction(x['Variables'], x['Coefficients']), axis=1)]
    print("Final Combination Shape:", results_df_filtered.shape)

    # Filter regression summaries based on the filtered results
    filtered_variables = results_df_filtered['Variables'].tolist()
    regression_summary_df_filtered = regression_summary_df[
        regression_summary_df['Variables'].apply(lambda x: x in filtered_variables)
    ]


    # Align indices with the order in results_df_filtered
    regression_summary_df_filtered = regression_summary_df_filtered.loc[results_df_filtered.index]
    
except Exception as e:
    print("Error during filtering:", e)

# Display filtered regression summaries
print("Filtered Regression Summaries DataFrame:",regression_summary_df_filtered.shape)


Total Combination 102050


Processing Time: 100%|██████████| 102050/102050 [36:43<00:00, 46.31it/s] 


Result_df Shape (12, 13)


100%|██████████| 12/12 [00:00<?, ?it/s]

Final Combination Shape: (9, 13)
Filtered Regression Summaries DataFrame: (9, 2)


In [19]:
results_df_filtered.head(4)

,Variables,Coefficients,Lower Estimate,Estimates,Upper Estimate,Sum of Estimates,R-squared,Adjusted R-squared,P-values,VIF,Durbin-Watson,Breusch-Pagan P Value,Mean Residuals
9,"[const, PD_BKT_0_Lag, Electricity , WPI, Sales...","[4.6362, 1.5579, -0.5422, 0.5649, -0.5226]",1.1890,2.8719,4.5549,8.6158,0.7706,0.7462,"[0.0, 0.0, 0.0089, 0.0241, 0.0053]","[1.0, 1.1158, 2.1175, 3.148, 1.7158]",1.9920,0.0163,2.429922e-16
8,"[const, PD_BKT_0_Lag, WPI, Sales_Passenger_Veh...","[4.6362, 1.3618, 0.4686, -0.8086]",1.3178,2.8010,4.2841,8.4029,0.7631,0.7434,"[0.0, 0.0, 0.0469, 0.0011]","[1.0, 1.4851, 2.7975, 2.8982]",1.7619,0.0051,-1.340647e-15
11,"[const, PD_BKT_0_Lag, WPI, Sales_Commerical_Ve...","[4.6362, 1.682, 0.5337, -1.0053, 0.5598]",1.6059,3.6188,5.6318,10.8565,0.7647,0.7397,"[0.0, 0.0, 0.0361, 0.0012, 0.0175]","[1.0, 1.0386, 3.1996, 4.425, 2.7028]",1.7510,0.0630,-1.206582e-15
4,"[const, PD_BKT_0_Lag, PMI]","[4.6362, 1.4092, -0.4826]",3.2904,3.5061,3.7219,10.5184,0.7433,0.7276,"[0.0, 0.0, 0.0049]","[1.0, 1.3399, 1.3399]",1.6791,0.0423,1.826631e-15


# Export

In [20]:
with pd.ExcelWriter(rf"{Folder_path}\{name} Possible All Combination.xlsx",engine='xlsxwriter') as want:
    df_info.to_excel(want,sheet_name="Inputs Section",index=False)
    results_df_filtered.to_excel(want,sheet_name="Final Combination",index=False)
    regression_summary_df_filtered.to_excel(want,sheet_name="Regression Summary",index=False)
    results_df.to_excel(want,sheet_name="Comb WO Correct direction",index=False)
    summary_df.to_excel(want,sheet_name="Mean Std",index=False)
    model_data.to_excel(want,sheet_name="Model Data",index=False)
    
    